In [4]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pandas as pd
import pickle
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [5]:
from batch_sv_system_utils import run_trial, plot_score, get_embeds

In [6]:
plda_model_dir = "embeddings/voxc2_fbank64_voxc2untied_tdnn7_embeds/plda_train/"

In [7]:
from utils import key2df, df2dict

si_keys = pickle.load(open("embeddings/voxc2_fbank64_voxc2untied_tdnn7_embeds/si_keys.pkl", "rb"))
si_embeds = np.load("embeddings/voxc2_fbank64_voxc2untied_tdnn7_embeds/si_embeds.npy")
si_key_df = key2df(si_keys)
si_id2idx, si_idx2id = df2dict(si_key_df) 

In [8]:
from utils import key2df, df2dict

sv_keys = pickle.load(open("embeddings/voxc2_fbank64_voxc2untied_tdnn7_embeds/sv_keys.pkl", "rb"))
sv_embeds = np.load("embeddings/voxc2_fbank64_voxc2untied_tdnn7_embeds/sv_embeds.npy")
sv_key_df = key2df(sv_keys)
sv_id2idx, sv_idx2id = df2dict(sv_key_df) 

In [9]:
large_si_spk = si_key_df.spk.value_counts().index[0:10]
cohort_ids = si_key_df[si_key_df.spk.isin(large_si_spk)].sample(n=5000).index
cohort_embeds = get_embeds(cohort_ids, si_embeds, si_id2idx, norm=False)

In [10]:
from batch_sv_system_utils import compute_eer

small_train_set_spks = si_key_df.spk.value_counts()[:10].index.tolist()
small_key_df = si_key_df[si_key_df.spk.isin(small_train_set_spks)].sample(n=3000)
small_embeds = get_embeds(small_key_df.index, si_embeds, si_id2idx, norm=False) 
scores = run_trial(small_embeds, small_embeds, np.random.randint(0, 2, size=(len(small_embeds))), 
                  plda_dir=plda_model_dir, plot=False)

label_mat = np.repeat(np.array(small_key_df.label).reshape(-1, 1), len(small_key_df), axis=1)
trial_labels = label_mat == label_mat.T

eer=0.5040, fpr=0.5040, fnr=0.5040


In [11]:
pos_scores = scores[np.nonzero(trial_labels)]
neg_scores = scores[np.nonzero(~trial_labels)]
compute_eer(pos_scores, neg_scores)

(0.03194806997130336,
 0.03194806997130336,
 0.03194515061157155,
 -7.4263770000000005)

In [67]:
from batch_sv_system_utils import compute_eer

small_train_set_spks = sv_key_df.spk.value_counts()[-30:].index.tolist()
small_key_df = sv_key_df[sv_key_df.spk.isin(small_train_set_spks)]
small_embeds = get_embeds(small_key_df.index, sv_embeds, sv_id2idx, norm=False) 
scores = run_trial(small_embeds, small_embeds, np.random.randint(0, 2, size=(len(small_embeds))), 
                  plda_dir=plda_model_dir, plot=False)

label_mat = np.repeat(np.array(small_key_df.label).reshape(-1, 1), len(small_key_df), axis=1)
trial_labels = label_mat == label_mat.T
pos_scores = scores[np.nonzero(trial_labels)]
neg_scores = scores[np.nonzero(~trial_labels)]

compute_eer(pos_scores, neg_scores)

eer=0.5049, fpr=0.5049, fnr=0.5045


(0.04127386145115142, 0.041261698724344244, 0.04127386145115142, -4.255052)

In [12]:
score_idx =np.arange(len(pos_scores)+len(neg_scores))
np.random.shuffle(score_idx)
pos_score_idx = score_idx[:len(pos_scores)]
neg_score_idx = score_idx[:len(neg_scores)]

In [ ]:
plt.figure(figsize=(20,5))
plt.scatter(pos_score_idx, pos_scores, alpha=0.5, color='r')
plt.scatter(neg_score_idx, neg_scores, alpha=0.5, color='b')

In [3]:
score_idx

NameError: name 'score_idx' is not defined